In [4]:
import pandas as pd
import numpy as np
import random
import os


from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [13]:
train = pd.read_csv('./train.csv')
#train = pd.read_csv('./train.csv').head(600000)
kosdaq_adr = pd.read_csv('./stock_data/kosdaq_adr.csv')
kospi_adr = pd.read_csv('./stock_data/kospi_adr.csv') 




,Unnamed: 0,Date,ADR
0,0,20180619,74.02
1,1,20180620,73.81
2,2,20180621,69.72
3,3,20180622,70.92
4,4,20180625,75.88
...,...,...,...
1245,1245,20230704,76.05
1246,1246,20230705,72.89
1247,1247,20230706,69.01
1248,1248,20230707,64.84


In [10]:
# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

# 볼린저밴드 상한선, 하한선, 밴드폭 계산 함수
Period = 60
Length = 20
StdDev = 1.5
def make_band(close):
    center = np.zeros(Length)
    for i in range(Length):
        center[i] = float(close[i])

    ma20 = np.mean(center)
    moving_std = np.std(center)
    BBTop = ma20 + (StdDev * moving_std)
    BBBot = ma20 - (StdDev * moving_std)
    Bwidth = (BBTop - BBBot) / ma20
    return BBTop, BBBot, Bwidth

# 이격도 전일과의 차이 계산 함수
def disparity(df):

    mean60 = np.zeros(Period)
    # 이동평균값 계산
    for i in range(Period):
        mean60[i] = float(df[i])
    ma60 = np.mean(mean60)

    # 이격도 계산
    dis60 = 100 * (df[i] / ma60)

    return dis60

####################################
# 날짜 종목코드, 종목명, 거래량, 시가 저가 고가 종가

## 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
#code = "A103840"
# # 학습 데이터 생성 : 등락에 가중치 부여
    train_vol = train[train['종목코드'] == code][['일자', '고가', '저가', '거래량']] # 거래량을 통하여 거래대금 계산 # 각 종목의 모든 날짜 데이터를 가져온다.
    train_close = train[train['종목코드'] == code][['일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)
    train_vol['일자'] = pd.to_datetime(train_vol['일자'], format='%Y%m%d')
    train_vol.set_index('일자', inplace=True)

    df_b = train_close['종가'].copy() # 볼린저밴드 밴드폭 가중치를 주기 위한 카피본
    dis = train_close['종가'].copy()  # 이격도 차이 가중치를 주기 위한 임시 카피본
    tp = train_close['종가'].copy()   # 거래대금 가중치를 주기 위한 임시 카피본
    dis.name = 'ma60'

    # 등락폭 계산
    train_close["shift"] = train_close["종가"].shift(1)
    train_close["가중치"] = ((train_close["종가"] - train_close["shift"]) / train_close["shift"])  # 당일종가-전일종가/전일종가 로 등락폭 계산
    train_close.drop("shift", axis=1)       #.to_clipboard(index=False)

    # 거래대금 계산
    tp = ((train_vol['고가'] + train_vol['저가']) / 2) * train_vol['거래량'] / 100000000000

    # 상 들어간 종목 외에는 가중치를 0.05로 고정 
    train_close[train_close['가중치'] < 0.285] = 0.05


    # 거래대금 250억에서 800억 사이에 종목 가중치 부여
    tp[(0.25 > tp) | (tp > 0.8)] = 0.0

    train_close['가중치'] += tp


    # 이격도 차이 배열 정리
    for i in range(len(df_b)+1):
        if i > 0 and i+59 < len(df_b):
            dis[-i-1] = disparity(df_b[-(60+i) : -i ])
        elif i == 0 :
            dis[-1] = disparity(df_b[-60 : ])
        else :
            dis[-i] = 0



    ## 이격도, 볼밴 상하한, 밴드폭을 이용하여 가중치 부여
    for i in range(len(df_b)+1):
        if i > 0 and i+19 < len(df_b):
            upb, downb, bw = make_band(df_b[ -(20+i) : -i ])

            if 0.15 <= bw and 0.85 >= bw and df_b[-i-1] > upb :
                train_close['가중치'].iloc[-i-1] += bw
            elif 0.15 <= bw and 0.85 >= bw and df_b[-i-1] < downb :
                train_close['가중치'].iloc[-i-1] -= bw



        elif i == 0 :
            upb, downb, bw = make_band(df_b[ -20 :  ])

            if 0.15 <= bw and 0.85 >= bw and df_b[-1] > upb :
                train_close['가중치'].iloc[-1] += bw
            elif 0.15 <= bw and 0.85 >= bw and df_b[-1] < downb :
                train_close['가중치'].iloc[-1] -= bw

        if i > 0 and i+59 < len(df_b):
            if dis[-i-1] >= dis[-i-2] + 10 :
                train_close['가중치'].iloc[-i-1] += (dis[-i-1] - dis[-i-2])/100
            elif dis[-i-1] <= dis[-i-2] - 7 :
                train_close['가중치'].iloc[-i-1] -= (dis[-i-2] - dis[-i-1])/100

        elif i == 0 :
            if dis[-1] >= dis[-2] + 10 :
                train_close['가중치'].iloc[-1] += (dis[-1] - dis[-2])/100
            elif dis[-1] <= dis[-2] - 7 :
                train_close['가중치'].iloc[-1] -= (dis[-2] - dis[-1])/100

    tc = train_close['가중치']  # 첫 번째 루프에 대한 데이터 약 400일정도



    ### 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측

    # 최종 수익률 계산
    final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]

    # 결과 저장
    results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)

100%|██████████| 2000/2000 [16:13<00:00,  2.05it/s]


In [19]:
predictions

300    0.050121
301    0.050168
302    0.050206
303    0.050232
304    0.050251
305    0.050263
306    0.050272
307    0.050278
308    0.050283
309    0.050286
310    0.050288
311    0.050289
312    0.050290
313    0.050291
314    0.050292
Name: predicted_mean, dtype: float64

In [ ]:
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format
results_df['final_return'] = results_df['final_return'].fillna(0)
results_df['final_return'] = abs(results_df['final_return'])
results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
results_df.sort_values(by='순위')

In [2]:
results_df.to_csv('base_submission.csv', index=False)

NameError: name 'results_df' is not defined

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')

In [ ]:
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')

In [ ]:
baseline_submission.to_csv('test_submission.csv', index=False)

In [ ]:
# 1460 / -604 ## 상따에 3배 가중치, 볼밴 <= 0.85 위아래 200위까지  2064  안뒤집었을 때 -

# 2374 / -2231 ## 상따, 볼밴 <= 1 위아래 200위  4605 이걸로 다시 제출   이게 0.53점(뒤집지 않았을 때)

# 88 / -44 # 상따만 했을 때     132 이게 0.23점